In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
path = os.getcwd()
print(path)

/Users/makkimaki/PycharmProjects/python_programming/SchrodingerSolver_HarmonicLikePotential1D


In [3]:
filename = "SymmetricPotentialEnergy.csv"
df = pd.read_csv(filename, header=4)

In [4]:
df.head()

,% dist,r,"Electric potential (V), Point: (0, 0)"
0,0.14,0.00,0.139774
1,0.14,0.02,0.139782
2,0.14,0.04,0.139795
3,0.14,0.06,0.139814
4,0.14,0.08,0.139829


In [5]:
df = df.rename(columns={"% dist" : 'z', 'Electric potential (V), Point: (0, 0)':'potential'})
df.head()

,z,r,potential
0,0.14,0.00,0.139774
1,0.14,0.02,0.139782
2,0.14,0.04,0.139795
3,0.14,0.06,0.139814
4,0.14,0.08,0.139829


In [8]:
z = df["z"]
r = df["r"]
potential = df["potential"]　
print("length of r:", r.shape)
print("length of z:", z.shape)
print("shape of potential:", potential.shape)

SyntaxError: invalid character in identifier (<ipython-input-8-568a6c42fdfc>, line 3)

In [ ]:
df_pivot = pd.pivot_table(data=df, index="z", columns="r", values="potential")
print("df_pivot shape:", df_pivot.shape,  "\n")
print("Table data: \n ", df_pivot)

In [ ]:
plt.pcolor(df_pivot.columns, df_pivot.index, df_pivot, cmap="jet")
plt.colorbar()
# plt.axis("tight")
plt.xlabel("r")
plt.ylabel("z")
plt.show()

In [ ]:
rmin = np.min(r)
rmax = np.max(r)
print("rmin=",rmin, " ","rmax=",rmax)
zmin = np.min(z)
zmax = np.max(z)
print("zmin=",zmin, " ","zmax=",zmax)

In [ ]:
mesh_number = 1000
N_r = len(r)
N_z = len(z)
print("N_r:", N_r)
print("N_z:", N_z)
x = np.linspace(rmin, rmax, num=mesh_number)
y = np.linspace(zmin, zmax, num=mesh_number)
xx, yy = np.meshgrid(x, y)

In [ ]:
import scipy.interpolate as interp

In [ ]:
fun = interp.interp2d(df_pivot.columns, df_pivot.index, df_pivot, kind="cubic")
# fun(1,3)[0]
funs = fun(x, y)

In [ ]:
plt.plot(x, fun(x, 0.15))
plt.xlabel("r ($\mu m$)")
plt.ylabel("Voltage (V)")
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

z_label = r"Voltage (V)"

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(xx, yy, funs, cmap='jet', linewidth=0) 
fig.colorbar(surf, label=z_label)
plt.xlabel("r ($\mu m$)")
plt.ylabel("z ($\mu m$)")
# ax.set_zlabel("Voltage (V)")


plt.show()

In [ ]:
#Physical quantity and parameter
shift = 3000
A = 20
k = 0.006951
a0 = 3.8066*2 # nm
re = 50/(A*a0/2)
h = 141/(A*a0/2)
hbarhbardevidedby2m = 9212.5 # GHz/nm^2

rmin = 0
zmin = 0
rmax = 2040/(A*a0/2)
zmax = zmin + 190/(A*a0/2) - h
x = np.linspace(-rmax, rmax, 1000) # unitless

# Define step size along coordinate
h = x[1] - x[0]
StateNumber = 20
N = mesh_number


# Create kinetic energy matrix
T = np.zeros((N-2)**2).reshape(N-2, N-2)
for i in range(N-2):
    for j in range(N-2):
        if i == j:
            T[i, j] = -2 
        elif i-j == 1:
            T[i, j] = 1 - 1/x[i] * h/2
        elif i-j == -1:
            T[i, j] == 1 + 1/x[i] * h/2
        else:
            T[i, j] = 0

# Create potential energy energy matrix
V = np.zeros((N-2)**2).reshape(N-2, N-2)
for i in range(N-2):
    for j in range(N-2):
        if i == j:
            V[i, j] = 1000 * 241.8 * (A*a0/2)**2 * fun((A*a0/2)*x[i+1]/1000, 0.15) / hbarhbardevidedby2m
        else:
            V[i, j] = 0

# Create Hamiltonian matrix
H = -T/(2*h**2) + V + shift

# Find eigenvalues and eigenvectors, then sort them in ascending order
val, vec = np.linalg.eig(H)
z = np.argsort(val - shift)
z = z[0 : StateNumber]
# energies = ((val[z] - shift) / (val[z][0] - shift))
energies = val[z] - shift
print(energies)


plt.figure(figsize=(6, 6))
for i in range(len(z)):
    y = []
    y = np.append(y, vec[:, z[i]])
    y = np.append(y, 0)
    y = np.insert(y, 0, 0)
    plt.plot(x*(A*a0/2), y, lw=3, label="{}".format(i))
    plt.ylabel("$psi$(x)", size = 14)

plt.title("normalized wave functions for a harmonic oscillator using finite difference method", size =14)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in range(0, StateNumber):
    plt.subplot(5,4, i+1)
    plt.title("State Number %d" % (i+1))
    y = []
    y = np.append(y, vec[:, z[i]])
    y = np.append(y, 0)
    y = np.insert(y, 0, 0)
    plt.plot(x*(A*a0/2), y, lw=1.5, label="{}".format(i))
#     plt.ylabel("$psi$(x)", size = 14)
plt.tight_layout()
plt.show()